# 傅里叶神经算子 (FNO) 教程

## 📚 简介

欢迎来到**傅里叶神经算子（Fourier Neural Operator, FNO）**的综合教程！

FNO 是一种强大的神经网络架构，通过在**傅里叶域**中操作，学习**无限维函数空间**之间的映射。与传统神经网络学习点到点的映射不同，FNO 学习的是**算子**——从函数到函数的映射。

### 🎯 FNO 的核心优势：
1. **分辨率独立性**：在一种网格上训练，在另一种网格上测试（零样本超分辨率）
2. **全局感受野**：每个输出都依赖于所有输入（通过谱卷积）
3. **计算高效**：由于使用 FFT，复杂度为 O(N log N)
4. **物理洞察**：在频域操作，自然捕获全局模式

### 📖 本教程内容：
- 傅里叶变换基础和直觉理解
- 谱卷积层
- 从零开始构建 FNO
- 在一维 Burgers 方程上训练
- 测试分辨率独立性
- 与传统求解器的比较

让我们开始吧！

## 1. 导入所需库

首先，让我们导入实现 FNO 所需的所有库。

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.axes_grid1 import make_axes_locatable
import time
from tqdm import tqdm

# 设置随机种子以保证可复现性
torch.manual_seed(42)
np.random.seed(42)

# 设备配置
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用设备: {device}")

# 绘图设置
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 4)
plt.rcParams['font.size'] = 10

## 2. 理解傅里叶变换

在深入 FNO 之前，让我们建立对**傅里叶变换**的直觉理解——这是 FNO 的数学基础。

### 什么是傅里叶变换？
傅里叶变换将信号分解为其组成频率：

$$\mathcal{F}[u](k) = \hat{u}(k) = \int_{-\infty}^{\infty} u(x) e^{-2\pi i k x} dx$$

对于离散信号，我们使用**快速傅里叶变换（FFT）**，它能在 O(N log N) 时间内完成计算。

### 为什么它对 PDE 有用？
许多偏微分方程在傅里叶空间中变成**代数方程**：
- 导数 → 乘法：$\frac{\partial u}{\partial x} \leftrightarrow 2\pi i k \hat{u}(k)$
- 卷积 → 乘法：$u * v \leftrightarrow \hat{u} \cdot \hat{v}$

让我们可视化一下！

In [ ]:
# 创建复合信号：多个正弦波的叠加
N = 256
x = np.linspace(0, 2*np.pi, N)

# 信号分量
f1, f2, f3 = 2, 5, 10  # 频率
u = np.sin(f1 * x) + 0.5 * np.sin(f2 * x) + 0.3 * np.sin(f3 * x)

# 计算 FFT
u_fft = np.fft.fft(u)
freqs = np.fft.fftfreq(N, x[1] - x[0])

# 绘制信号及其频谱
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# 原始信号
axes[0].plot(x, u, 'b-', linewidth=2)
axes[0].set_title('空间域信号', fontsize=14, fontweight='bold')
axes[0].set_xlabel('x')
axes[0].set_ylabel('u(x)')
axes[0].grid(True, alpha=0.3)

# 频谱（幅值）
axes[1].stem(freqs[:N//2], np.abs(u_fft[:N//2]), linefmt='r-', markerfmt='ro', basefmt=' ')
axes[1].set_title('频谱（幅值）', fontsize=14, fontweight='bold')
axes[1].set_xlabel('频率 (k)')
axes[1].set_ylabel('|û(k)|')
axes[1].set_xlim(-1, 15)
axes[1].grid(True, alpha=0.3)

# 从 FFT 重建信号
u_reconstructed = np.fft.ifft(u_fft).real
axes[2].plot(x, u, 'b-', label='原始信号', linewidth=2, alpha=0.7)
axes[2].plot(x, u_reconstructed, 'r--', label='重建信号', linewidth=2)
axes[2].set_title('从 FFT 重建', fontsize=14, fontweight='bold')
axes[2].set_xlabel('x')
axes[2].set_ylabel('u(x)')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"重建误差: {np.max(np.abs(u - u_reconstructed)):.2e}")

## 3. 实现谱卷积层

**谱卷积**是 FNO 的核心创新。与在空间域卷积不同，我们：
1. 通过 FFT 将输入转换到傅里叶空间
2. 与可学习的权重相乘（仅保留低频模态）
3. 通过逆 FFT 转换回空间域

### 数学表述：
$$(K * u)(x) = \mathcal{F}^{-1}\left(R_{\theta} \cdot \mathcal{F}[u]\right)(x)$$

其中 $R_{\theta}$ 是可学习的复值权重。

### 为什么截断到低频模态？
- **计算效率**：更少的参数
- **正则化**：防止过拟合高频噪声
- **物理解释**：大多数 PDE 解由低频主导

In [ ]:
class SpectralConv1d(nn.Module):
    """FNO 的一维谱卷积层"""
    
    def __init__(self, in_channels, out_channels, modes):
        """
        参数:
            in_channels: 输入通道数
            out_channels: 输出通道数
            modes: 保留的傅里叶模态数（低频模态）
        """
        super(SpectralConv1d, self).__init__()
        
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.modes = modes  # 要相乘的傅里叶模态数
        
        # 初始化缩放因子
        self.scale = 1 / (in_channels * out_channels)
        
        # 傅里叶空间中的可学习权重（复值）
        # 形状: [in_channels, out_channels, modes]
        self.weights = nn.Parameter(
            self.scale * torch.rand(in_channels, out_channels, self.modes, dtype=torch.cfloat)
        )
    
    def compl_mul1d(self, input, weights):
        """傅里叶空间中的复数乘法"""
        # (batch, in_channel, x), (in_channel, out_channel, x) -> (batch, out_channel, x)
        return torch.einsum("bix,iox->box", input, weights)
    
    def forward(self, x):
        """
        参数:
            x: 输入张量，形状为 (batch, in_channels, n)
        返回:
            输出张量，形状为 (batch, out_channels, n)
        """
        batch_size = x.shape[0]
        
        # 1. 使用 FFT 计算傅里叶系数
        # 对实值输入使用 rfft（比 fft 更高效）
        x_ft = torch.fft.rfft(x, dim=-1)
        
        # 2. 将相关的傅里叶模态与可学习权重相乘
        out_ft = torch.zeros(batch_size, self.out_channels, x.size(-1)//2 + 1, 
                            dtype=torch.cfloat, device=x.device)
        
        # 仅乘以低频模态
        out_ft[:, :, :self.modes] = self.compl_mul1d(
            x_ft[:, :, :self.modes], self.weights
        )
        
        # 3. 通过逆 FFT 返回物理空间
        x = torch.fft.irfft(out_ft, n=x.size(-1), dim=-1)
        
        return x

# 测试谱卷积层
modes = 12
in_channels, out_channels = 2, 4
batch_size, n_points = 8, 64

spectral_conv = SpectralConv1d(in_channels, out_channels, modes).to(device)
x_test = torch.randn(batch_size, in_channels, n_points).to(device)
output = spectral_conv(x_test)

print(f"输入形状: {x_test.shape}")
print(f"输出形状: {output.shape}")
print(f"参数数量: {sum(p.numel() for p in spectral_conv.parameters())}")

## 4. 构建简单的 FNO 模块

一个 FNO 模块结合了：
1. **谱卷积**：傅里叶空间中的全局变换
2. **逐点线性层**：空间域中的局部变换
3. **激活函数**：非线性（通常使用 GELU）

### 一个 FNO 层的架构：
$$v_{l+1}(x) = \sigma\left(W v_l(x) + (K * v_l)(x)\right)$$

其中：
- $K * v_l$ 是谱卷积（全局）
- $W v_l$ 是逐点卷积（局部）
- $\sigma$ 是激活函数

In [ ]:
class FNOBlock1d(nn.Module):
    """结合谱卷积和逐点操作的单个 FNO 模块"""
    
    def __init__(self, width, modes):
        """
        参数:
            width: 通道数（隐藏维度）
            modes: 使用的傅里叶模态数
        """
        super(FNOBlock1d, self).__init__()
        
        # 谱卷积（全局）
        self.spectral_conv = SpectralConv1d(width, width, modes)
        
        # 逐点卷积（局部）- 实现为 1x1 卷积
        self.pointwise_conv = nn.Conv1d(width, width, 1)
        
        # 激活函数
        self.activation = nn.GELU()
    
    def forward(self, x):
        """
        参数:
            x: 输入张量，形状为 (batch, width, n)
        返回:
            输出张量，形状为 (batch, width, n)
        """
        # 全局分支：谱卷积
        x1 = self.spectral_conv(x)
        
        # 局部分支：逐点卷积
        x2 = self.pointwise_conv(x)
        
        # 组合并激活
        x = self.activation(x1 + x2)
        
        return x

# 测试 FNO 模块
width, modes = 32, 12
n_points = 64

fno_block = FNOBlock1d(width, modes).to(device)
x_test = torch.randn(4, width, n_points).to(device)
output = fno_block(x_test)

print(f"输入形状: {x_test.shape}")
print(f"输出形状: {output.shape}")
print(f"形状保持: {output.shape == x_test.shape}")

## 5. 创建完整的 FNO 架构

现在让我们构建完整的 FNO 网络！架构包括：

1. **提升层 (Lifting Layer, P)**：将输入从低维提升到高维表示
   - 映射 $u_0: \mathbb{R}^{d_in} \to \mathbb{R}^{width}$

2. **FNO 层**：L 个 FNO 模块的堆叠
   - 每个都应用谱卷积 + 逐点卷积 + 激活

3. **投影层 (Projection Layer, Q)**：投影回输出维度
   - 映射 $v_L: \mathbb{R}^{width} \to \mathbb{R}^{d_{out}}$

### 完整的 FNO 流程：
$$\text{FNO}(u) = Q \circ \sigma(W_L + K_L) \circ \cdots \circ \sigma(W_1 + K_1) \circ P(u)$$

In [ ]:
class FNO1d(nn.Module):
    """完整的一维傅里叶神经算子"""
    
    def __init__(self, modes, width, n_layers=4, in_channels=1, out_channels=1):
        """
        参数:
            modes: 使用的傅里叶模态数
            width: 隐藏维度（通道数）
            n_layers: FNO 模块数量
            in_channels: 输入通道数
            out_channels: 输出通道数
        """
        super(FNO1d, self).__init__()
        
        self.modes = modes
        self.width = width
        self.n_layers = n_layers
        
        # 提升层：将输入映射到更高维空间
        self.lifting = nn.Conv1d(in_channels, width, 1)
        
        # FNO 模块
        self.fno_blocks = nn.ModuleList([
            FNOBlock1d(width, modes) for _ in range(n_layers)
        ])
        
        # 投影层：映射回输出维度
        self.projection = nn.Sequential(
            nn.Conv1d(width, 128, 1),
            nn.GELU(),
            nn.Conv1d(128, out_channels, 1)
        )
    
    def forward(self, x):
        """
        参数:
            x: 输入张量，形状为 (batch, in_channels, n)
        返回:
            输出张量，形状为 (batch, out_channels, n)
        """
        # 提升到更高维度
        x = self.lifting(x)  # (batch, in_channels, n) -> (batch, width, n)
        
        # 应用 FNO 模块
        for block in self.fno_blocks:
            x = block(x)  # (batch, width, n) -> (batch, width, n)
        
        # 投影到输出维度
        x = self.projection(x)  # (batch, width, n) -> (batch, out_channels, n)
        
        return x

# 创建 FNO 模型
modes = 16
width = 64
n_layers = 4

model = FNO1d(modes=modes, width=width, n_layers=n_layers, 
              in_channels=1, out_channels=1).to(device)

# 统计参数
total_params = sum(p.numel() for p in model.parameters())
print(f"✅ FNO 模型已创建！")
print(f"   - 模态数: {modes}")
print(f"   - 宽度: {width}")
print(f"   - 层数: {n_layers}")
print(f"   - 总参数量: {total_params:,}")

# 测试前向传播
x_test = torch.randn(4, 1, 256).to(device)
output = model(x_test)
print(f"\n🔍 前向传播测试:")
print(f"   输入形状: {x_test.shape}")
print(f"   输出形状: {output.shape}")

## 6. 生成合成 PDE 数据

现在让我们生成训练数据！我们将使用**一维 Burgers 方程**：

$$\frac{\partial u}{\partial t} + u \frac{\partial u}{\partial x} = \nu \frac{\partial^2 u}{\partial x^2}$$

这是一个基础的非线性 PDE，用于模拟：
- 流体动力学（简化的 Navier-Stokes）
- 交通流
- 激波传播

### 我们的任务：
学习解算子：$G_\nu: u_0(x) \mapsto u(x, T)$
- 输入：$t=0$ 时刻的初始条件 $u_0(x)$
- 输出：最终时刻 $T$ 的解 $u(x, T)$
- 参数：粘度 $\nu$（控制扩散）

In [ ]:
def solve_burgers_spectral(u0, nu, T, n_steps=1000):
    """
    使用谱方法求解一维 Burgers 方程（伪谱法）
    
    参数:
        u0: 初始条件（numpy 数组）
        nu: 粘度系数
        T: 最终时间
        n_steps: 时间步数
        
    返回:
        时刻 T 的解
    """
    N = len(u0)
    L = 2 * np.pi
    x = np.linspace(0, L, N, endpoint=False)
    k = np.fft.fftfreq(N, L / (2*np.pi*N))
    
    dt = T / n_steps
    u = u0.copy()
    
    # 显式格式的时间步进
    for _ in range(n_steps):
        # 转换到傅里叶空间
        u_hat = np.fft.fft(u)
        
        # 在傅里叶空间计算导数
        u_x_hat = 1j * k * u_hat
        u_x = np.fft.ifft(u_x_hat).real
        
        # 物理空间中的非线性项
        nonlinear = -u * u_x
        
        # 傅里叶空间中的扩散项
        diffusion_hat = -nu * (k**2) * u_hat
        diffusion = np.fft.ifft(diffusion_hat).real
        
        # 更新（演示用的简单前向欧拉法）
        u = u + dt * (nonlinear + diffusion)
    
    return u

# 生成数据集
def generate_burgers_dataset(n_samples, n_points, nu=0.01, T=1.0):
    """生成 Burgers 方程解的数据集"""
    
    X_data = []  # 初始条件
    Y_data = []  # 时刻 T 的解
    
    L = 2 * np.pi
    x = np.linspace(0, L, n_points, endpoint=False)
    
    for i in tqdm(range(n_samples), desc="生成数据"):
        # 随机初始条件：正弦波的叠加
        n_modes = np.random.randint(3, 8)
        u0 = np.zeros(n_points)
        
        for _ in range(n_modes):
            freq = np.random.randint(1, 6)
            amp = np.random.uniform(0.5, 2.0)
            phase = np.random.uniform(0, 2*np.pi)
            u0 += amp * np.sin(freq * x + phase)
        
        # 求解 PDE
        u_T = solve_burgers_spectral(u0, nu, T, n_steps=1000)
        
        X_data.append(u0)
        Y_data.append(u_T)
    
    return np.array(X_data), np.array(Y_data), x

# 生成训练和测试数据
print("生成训练数据...")
n_train = 1000
n_test = 200
n_points = 256
nu = 0.01
T = 1.0

X_train, Y_train, x_grid = generate_burgers_dataset(n_train, n_points, nu, T)
X_test, Y_test, _ = generate_burgers_dataset(n_test, n_points, nu, T)

print(f"\n✅ 数据集已生成！")
print(f"   训练样本: {n_train}")
print(f"   测试样本: {n_test}")
print(f"   网格点数: {n_points}")
print(f"   粘度: {nu}")
print(f"   最终时间: {T}")

# 可视化一些样例
fig, axes = plt.subplots(2, 3, figsize=(15, 8))
for i in range(3):
    idx = np.random.randint(0, n_train)
    
    axes[0, i].plot(x_grid, X_train[idx], 'b-', linewidth=2, label='初始条件 u₀(x)')
    axes[0, i].set_title(f'样本 {i+1}: 初始条件', fontweight='bold')
    axes[0, i].set_xlabel('x')
    axes[0, i].set_ylabel('u')
    axes[0, i].grid(True, alpha=0.3)
    axes[0, i].legend()
    
    axes[1, i].plot(x_grid, Y_train[idx], 'r-', linewidth=2, label=f'解 u(x,T={T})')
    axes[1, i].set_title(f'样本 {i+1}: T={T} 时刻的解', fontweight='bold')
    axes[1, i].set_xlabel('x')
    axes[1, i].set_ylabel('u')
    axes[1, i].grid(True, alpha=0.3)
    axes[1, i].legend()

plt.tight_layout()
plt.show()

## 7. 在一维 Burgers 方程上训练 FNO

开始训练我们的 FNO 模型！我们将使用：
- **损失函数**：空间域的相对 L2 误差
- **优化器**：带学习率调度的 Adam
- **批量训练**：同时处理多个样本

### 相对 L2 误差：
$$\text{Loss} = \frac{\|u_{pred} - u_{true}\|_2}{\|u_{true}\|_2}$$

这通过解的幅度归一化，使其与尺度无关。

In [ ]:
# 准备数据加载器
class BurgersDataset(Dataset):
    def __init__(self, X, Y):
        self.X = torch.FloatTensor(X).unsqueeze(1)  # 添加通道维度
        self.Y = torch.FloatTensor(Y).unsqueeze(1)
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

train_dataset = BurgersDataset(X_train, Y_train)
test_dataset = BurgersDataset(X_test, Y_test)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 初始化模型、损失函数、优化器
model = FNO1d(modes=16, width=64, n_layers=4).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.5)

def relative_l2_error(pred, true):
    """计算相对 L2 误差"""
    return torch.norm(pred - true, 2) / torch.norm(true, 2)

# 训练循环
n_epochs = 100
train_losses = []
test_losses = []

print("🚀 开始训练...\n")

for epoch in range(n_epochs):
    # 训练
    model.train()
    train_loss = 0.0
    
    for x_batch, y_batch in train_loader:
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        
        optimizer.zero_grad()
        y_pred = model(x_batch)
        loss = relative_l2_error(y_pred, y_batch)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
    
    train_loss /= len(train_loader)
    train_losses.append(train_loss)
    
    # 验证
    model.eval()
    test_loss = 0.0
    
    with torch.no_grad():
        for x_batch, y_batch in test_loader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            y_pred = model(x_batch)
            loss = relative_l2_error(y_pred, y_batch)
            test_loss += loss.item()
    
    test_loss /= len(test_loader)
    test_losses.append(test_loss)
    
    scheduler.step()
    
    if (epoch + 1) % 10 == 0:
        print(f"轮次 [{epoch+1}/{n_epochs}] - 训练损失: {train_loss:.6f}, 测试损失: {test_loss:.6f}")

print("\n✅ 训练完成！")

# 绘制训练曲线
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='训练损失', linewidth=2)
plt.plot(test_losses, label='测试损失', linewidth=2)
plt.xlabel('轮次', fontsize=12)
plt.ylabel('相对 L2 误差', fontsize=12)
plt.title('训练进度', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.yscale('log')
plt.tight_layout()
plt.show()

## 8. 可视化预测与真实值

让我们看看训练好的 FNO 在测试数据上的表现如何！

In [ ]:
# 在测试集上评估
model.eval()

# 选择随机测试样本
n_samples = 4
indices = np.random.choice(len(X_test), n_samples, replace=False)

fig, axes = plt.subplots(n_samples, 3, figsize=(15, 4*n_samples))

with torch.no_grad():
    for i, idx in enumerate(indices):
        # 获取数据
        x_input = torch.FloatTensor(X_test[idx:idx+1]).unsqueeze(1).to(device)
        y_true = Y_test[idx]
        
        # 预测
        y_pred = model(x_input).squeeze().cpu().numpy()
        
        # 计算误差
        error = np.abs(y_pred - y_true)
        rel_error = np.linalg.norm(error) / np.linalg.norm(y_true) * 100
        
        # 绘制初始条件
        axes[i, 0].plot(x_grid, X_test[idx], 'b-', linewidth=2)
        axes[i, 0].set_title(f'样本 {i+1}: 初始条件 u₀(x)', fontweight='bold')
        axes[i, 0].set_xlabel('x')
        axes[i, 0].set_ylabel('u')
        axes[i, 0].grid(True, alpha=0.3)
        
        # 绘制预测与真实值
        axes[i, 1].plot(x_grid, y_true, 'g-', linewidth=2, label='真实值', alpha=0.7)
        axes[i, 1].plot(x_grid, y_pred, 'r--', linewidth=2, label='FNO 预测')
        axes[i, 1].set_title(f'样本 {i+1}: 预测 vs 真实值 (误差: {rel_error:.2f}%)', fontweight='bold')
        axes[i, 1].set_xlabel('x')
        axes[i, 1].set_ylabel('u')
        axes[i, 1].legend()
        axes[i, 1].grid(True, alpha=0.3)
        
        # 绘制绝对误差
        axes[i, 2].plot(x_grid, error, 'purple', linewidth=2)
        axes[i, 2].fill_between(x_grid, 0, error, alpha=0.3, color='purple')
        axes[i, 2].set_title(f'样本 {i+1}: 绝对误差', fontweight='bold')
        axes[i, 2].set_xlabel('x')
        axes[i, 2].set_ylabel('|u_pred - u_true|')
        axes[i, 2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 计算整体测试误差
test_errors = []
with torch.no_grad():
    for x_batch, y_batch in test_loader:
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        y_pred = model(x_batch)
        
        for i in range(len(x_batch)):
            error = relative_l2_error(y_pred[i], y_batch[i]).item() * 100
            test_errors.append(error)

print(f"\n📊 测试集性能:")
print(f"   平均误差: {np.mean(test_errors):.3f}%")
print(f"   标准差: {np.std(test_errors):.3f}%")
print(f"   最大误差: {np.max(test_errors):.3f}%")
print(f"   最小误差: {np.min(test_errors):.3f}%")

## 9. 测试分辨率独立性

FNO 最强大的特性之一是**分辨率独立性**：能够在一种网格分辨率上训练，然后在完全不同的分辨率上测试，**无需重新训练**！

### 谱方法的魔力：
- 传统 CNN：固定于训练分辨率（必须为不同网格重新训练）
- FNO：在频域操作 → 自然处理不同分辨率
- 这实现了**零样本超分辨率**

让我们通过以下方式测试：
1. 在 64 个点上训练（粗网格）
2. 在 256 个点上测试（细网格）← **4倍分辨率！**

In [ ]:
# 在不同分辨率下生成数据
print("🔬 测试分辨率独立性\n")

# 用于训练的粗分辨率
n_train_res = 100
n_coarse = 64
print(f"在粗分辨率（{n_coarse} 个点）生成训练数据...")
X_train_coarse, Y_train_coarse, x_coarse = generate_burgers_dataset(n_train_res, n_coarse, nu, T)

# 用于测试的细分辨率
n_test_res = 50
n_fine = 256
print(f"在细分辨率（{n_fine} 个点）生成测试数据...")
X_test_fine, Y_test_fine, x_fine = generate_burgers_dataset(n_test_res, n_fine, nu, T)

# 创建数据集
train_dataset_coarse = BurgersDataset(X_train_coarse, Y_train_coarse)
test_dataset_fine = BurgersDataset(X_test_fine, Y_test_fine)

train_loader_coarse = DataLoader(train_dataset_coarse, batch_size=32, shuffle=True)
test_loader_fine = DataLoader(test_dataset_fine, batch_size=32, shuffle=False)

# 在粗分辨率上训练新模型
print(f"\n🏋️ 在 {n_coarse} 点网格上训练 FNO...")
model_coarse = FNO1d(modes=12, width=48, n_layers=4).to(device)
optimizer = torch.optim.Adam(model_coarse.parameters(), lr=0.001)

n_epochs_quick = 50
for epoch in range(n_epochs_quick):
    model_coarse.train()
    for x_batch, y_batch in train_loader_coarse:
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        y_pred = model_coarse(x_batch)
        loss = relative_l2_error(y_pred, y_batch)
        loss.backward()
        optimizer.step()
    
    if (epoch + 1) % 10 == 0:
        print(f"  轮次 {epoch+1}/{n_epochs_quick}")

print(f"\n🧪 在 {n_fine} 点网格上测试（4倍精细！）...")

# 在细分辨率上测试
model_coarse.eval()
test_errors_fine = []

with torch.no_grad():
    for x_batch, y_batch in test_loader_fine:
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        y_pred = model_coarse(x_batch)
        
        for i in range(len(x_batch)):
            error = relative_l2_error(y_pred[i], y_batch[i]).item() * 100
            test_errors_fine.append(error)

print(f"\n✅ 分辨率独立性测试结果:")
print(f"   训练分辨率: {n_coarse} 个点")
print(f"   测试分辨率: {n_fine} 个点（4倍分辨率）")
print(f"   平均误差: {np.mean(test_errors_fine):.3f}%")
print(f"   标准差: {np.std(test_errors_fine):.3f}%")

# 可视化分辨率独立性
idx = 0
x_input_fine = torch.FloatTensor(X_test_fine[idx:idx+1]).unsqueeze(1).to(device)
y_true_fine = Y_test_fine[idx]

with torch.no_grad():
    y_pred_fine = model_coarse(x_input_fine).squeeze().cpu().numpy()

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].plot(x_fine, X_test_fine[idx], 'b-', linewidth=2)
axes[0].set_title(f'{n_fine} 个点的初始条件', fontweight='bold')
axes[0].set_xlabel('x')
axes[0].set_ylabel('u₀')
axes[0].grid(True, alpha=0.3)

axes[1].plot(x_fine, y_true_fine, 'g-', linewidth=2, label='真实值', alpha=0.7)
axes[1].plot(x_fine, y_pred_fine, 'r--', linewidth=2, label=f'FNO（在{n_coarse}点上训练）')
axes[1].set_title(f'在 {n_fine} 点网格上的预测', fontweight='bold')
axes[1].set_xlabel('x')
axes[1].set_ylabel('u')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

error = np.abs(y_pred_fine - y_true_fine)
axes[2].plot(x_fine, error, 'purple', linewidth=2)
axes[2].fill_between(x_fine, 0, error, alpha=0.3, color='purple')
axes[2].set_title('绝对误差', fontweight='bold')
axes[2].set_xlabel('x')
axes[2].set_ylabel('|误差|')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n🎉 FNO 成功实现了零样本超分辨率！")

## 10. 与传统方法的比较

让我们将 FNO 与传统的有限差分求解器进行比较，看看计算优势。

### 比较指标：
1. **推理速度**：获得解的速度有多快？
2. **精度**：与真实解有多接近？
3. **可扩展性**：性能如何随分辨率扩展？

传统方法在时间上逐步求解 PDE，而 FNO 学习从初始状态到最终状态的**直接映射**！

In [ ]:
# 速度比较
print("⚡ 速度比较: FNO vs 传统求解器\n")

n_samples = 50
u0_samples = X_test[:n_samples]

# 测试传统求解器的时间
print("测试传统谱方法求解器...")
start_time = time.time()
for u0 in u0_samples:
    _ = solve_burgers_spectral(u0, nu, T, n_steps=1000)
traditional_time = time.time() - start_time

# 测试 FNO 的时间
print("测试 FNO 推理...")
model.eval()
with torch.no_grad():
    start_time = time.time()
    x_batch = torch.FloatTensor(u0_samples).unsqueeze(1).to(device)
    _ = model(x_batch)
    fno_time = time.time() - start_time

print(f"\n📊 速度结果（{n_samples} 个样本）:")
print(f"   传统求解器: {traditional_time:.4f} 秒 ({traditional_time/n_samples*1000:.2f} 毫秒/样本)")
print(f"   FNO: {fno_time:.4f} 秒 ({fno_time/n_samples*1000:.2f} 毫秒/样本)")
print(f"   加速: {traditional_time/fno_time:.1f}x 更快！🚀")

# 精度比较
print("\n📏 精度比较:")
traditional_errors = []
fno_errors = []

model.eval()
with torch.no_grad():
    for i in range(n_samples):
        u0 = X_test[i]
        y_true = Y_test[i]
        
        # 传统求解器
        y_traditional = solve_burgers_spectral(u0, nu, T, n_steps=1000)
        trad_error = np.linalg.norm(y_traditional - y_true) / np.linalg.norm(y_true) * 100
        traditional_errors.append(trad_error)
        
        # FNO
        x_input = torch.FloatTensor(u0).unsqueeze(0).unsqueeze(0).to(device)
        y_fno = model(x_input).squeeze().cpu().numpy()
        fno_error = np.linalg.norm(y_fno - y_true) / np.linalg.norm(y_true) * 100
        fno_errors.append(fno_error)

print(f"   传统求解器误差: {np.mean(traditional_errors):.3f}% ± {np.std(traditional_errors):.3f}%")
print(f"   FNO 误差: {np.mean(fno_errors):.3f}% ± {np.std(fno_errors):.3f}%")

# 可视化
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 速度比较
methods = ['传统\n求解器', 'FNO']
times = [traditional_time/n_samples*1000, fno_time/n_samples*1000]
colors = ['#FF6B6B', '#4ECDC4']

bars = axes[0].bar(methods, times, color=colors, alpha=0.7, edgecolor='black', linewidth=2)
axes[0].set_ylabel('每样本时间 (毫秒)', fontsize=12)
axes[0].set_title('推理速度比较', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3, axis='y')

# 在柱状图上添加数值标签
for i, (bar, time_val) in enumerate(zip(bars, times)):
    height = bar.get_height()
    axes[0].text(bar.get_x() + bar.get_width()/2., height,
                f'{time_val:.2f} ms',
                ha='center', va='bottom', fontsize=11, fontweight='bold')

# 误差分布比较
axes[1].violinplot([traditional_errors, fno_errors], positions=[1, 2], 
                   showmeans=True, showmedians=True)
axes[1].set_xticks([1, 2])
axes[1].set_xticklabels(['传统\n求解器', 'FNO'])
axes[1].set_ylabel('相对误差 (%)', fontsize=12)
axes[1].set_title('精度比较', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\n" + "="*60)
print("🎯 关键要点:")
print("="*60)
print(f"✅ FNO 比传统求解器快 {traditional_time/fno_time:.0f}x")
print(f"✅ FNO 保持了竞争性的精度（{np.mean(fno_errors):.2f}% 误差）")
print(f"✅ FNO 具有分辨率独立性（零样本超分辨率）")
print(f"✅ FNO 学习解算子（无需时间步进）")
print("="*60)

## 🎓 总结与核心概念

### 我们学到了什么：

#### 1. **傅里叶神经算子基础**
- FNO 学习函数空间之间的算子（而不仅仅是点映射）
- 在傅里叶域操作以获得全局感受野
- 结合谱卷积（全局）+ 逐点卷积（局部）

#### 2. **核心组件**
- **谱卷积**：傅里叶空间中的可学习线性算子
  - FFT → 与权重相乘 → 逆 FFT
  - 仅保留低频模态（正则化 + 效率）
- **FNO 模块**：谱卷积 + 逐点卷积 + 激活
- **架构**：提升 → FNO 模块 → 投影

#### 3. **相比传统方法的优势**
- ✅ **速度**：比 PDE 求解器快 10-1000x
- ✅ **分辨率独立性**：在一种网格上训练，在任意网格上测试
- ✅ **全局信息流**：每个输出都依赖于所有输入
- ✅ **从数据学习**：无需推导数值格式

#### 4. **实践考虑**
- **模态数**：控制容量 vs. 正则化
- **宽度**：控制表达能力（通常 32-128）
- **深度**：更多层捕获更复杂的模式（通常 4-8）
- **损失函数**：相对 L2 误差是 PDE 的标准

### 📚 延伸阅读：
1. FNO 原始论文：[Li et al. 2020](https://arxiv.org/abs/2010.08895)
2. 物理信息 FNO：[Li et al. 2021](https://arxiv.org/abs/2111.03794)
3. 应用：天气预报、湍流建模、材料科学

### 🚀 下一步：
- 扩展到 2D/3D 问题（Navier-Stokes、Maxwell 方程）
- 尝试不同的 PDE（扩散、波动、反应扩散）
- 实现多尺度架构（U-FNO）
- 结合物理信息约束

---

**恭喜！** 🎉 你已经从零开始构建了完整的傅里叶神经算子，并将其应用于求解 PDE。你在这里学到的技术代表了科学机器学习的前沿！